In [1]:
import pyomo.environ as pyo
import pandas as pd

from plant_con.models.plant_model_single_pyo import Plant as PlantSingle
from plant_con.models.plant_model_recourse_pyo import Plant as PlantRecourse


# Problem Overview

The model is a (deliberately non-realistic) refinery model that takes in crude oil (light or heavy) and produces light, medium, and heavy products.
The refinery has a distillation unit that can take in a certain amount of crude oil to produce light, medium, and heavy intermediates, and three refining units that can take in intermediate products to produce the final products.
The light and medium refining units can take in both light and medium intermediates, while the heavy refining unit can only take in medium and heavy intermediates. The ratios of inputs to intermediates, and intermediates to outputs are parameters passed to the constructor of the model.

The model is designed to optimize the production of the final products given the constraints of the distillation and refining units as well as:
- the ratios of crude oil to intermediate products
- the ratios of intermediate products to final products.
- the prices of the inputs (light/heavy crude oil) and outputs.
- The "demands" for the final products: above a certain amount, the product no longer has value.

## Recourse vs Non-Recourse
The first version of the model is a single optimization problem that assumes perfect foresight of the prices and demands for the final products.

The second version of the model is a recourse model that solves the same problem, but with multiple scenarios of prices and demands for the final products.
The recourse model assumes that the quantities of inputs (light/heavy crude) must be decided in advance of knowing the prices and demands for the final products, but that the choice to allocate the intermediate products to the refining units can be made after the prices and demands are known (this is the "recourse" decision).
The recourse model treats all the scenarios as equally likely.

In [2]:
# Define a helper function to determine the objective function for a given scenario of the recourse model
def scenario_obj(
    m,
    i: int,
    prod_light_price: list[int],
    prod_medium_price: list[int],
    prod_heavy_price: list[int],
    crude_light_price: int,
    crude_heavy_price: int,
) -> float:
    """
    Determine the objective function for a given scenario of the recourse model
    - the real objective includes all the scenarios combined, so use this for comparison
    """
    v = pyo.value

    return (
        v(m.light_prod_full_price[i]) * prod_light_price[i]
        + v(m.med_prod_full_price[i]) * prod_medium_price[i]
        + v(m.heavy_prod_full_price[i]) * prod_heavy_price[i]
        - v(m.light_crude_import) * crude_light_price
        - v(m.heavy_crude_import) * crude_heavy_price
    )

# Scenarios

The scenarios to be tested are:
0 - light demand is zero, medium and heavy demand high (2000 - higher than achievable)
1 - light demand is 50, medium demand is 50, heavy demand is high
2 - all demands high

In [3]:
# Scenarios to test in optimization:

# constants
crude_light_ratios = (3, 0.3, 0)
crude_heavy_ratios = (0, 1, 4)
light_product_ratios = (2, 1)
medium_product_input_ratios = (1, 1)
heavy_product_input_ratios = (1, 2)

crude_light_price = 30
crude_heavy_price = 10

# Define values
# capacity
c = 2000
crude_cap = 100

scenarios = 3
prod_light_price = [50] * 3
prod_light_demand = [0, 50, c]
prod_medium_price = [10] * 3
prod_medium_demand = [c, 50, c]
prod_heavy_price = [10] * 3
prod_heavy_demand = [c, c, c]

In [4]:
opt = pyo.SolverFactory("glpk")

In [5]:
# Save the results of the non-recourse solutions
non_recourse_df = pd.DataFrame(
        columns=[
            "Scenario",
            "Objective Value",
            "Light Crude Import",
            "Heavy Crude Import",
            "Light Product Output",
            "Medium Product Output",
            "Heavy Product Output",
            "Light Inter to Light Unit",
            "Med Inter to Light Unit",
            "Light Inter to Med Unit",
            "Med Inter to Med Unit",
            "Med Inter to Heavy Unit",
            "Heavy Inter to Heavy Unit",
        ]
    )

In [6]:
for ii in range(scenarios):
    p = PlantSingle(
        crude_distil_cap=crude_cap,
        crude_light_ratios=crude_light_ratios,
        crude_heavy_ratios=crude_heavy_ratios,
        refine_light_cap=c,
        refine_medium_cap=c,
        refine_heavy_cap=c,
        prod_light_ratios=light_product_ratios,
        prod_medium_ratios=medium_product_input_ratios,
        prod_heavy_ratios=heavy_product_input_ratios,
        crude_light_price=crude_light_price,
        crude_heavy_price=crude_heavy_price,
        prod_light_price=prod_light_price[ii],
        prod_light_demand=prod_light_demand[ii],
        prod_medium_price=prod_medium_price[ii],
        prod_medium_demand=prod_medium_demand[ii],
        prod_heavy_price=prod_heavy_price[ii],
        prod_heavy_demand=prod_heavy_demand[ii],
    )

    opt.solve(p.model)

    non_recourse_df.loc[ii] = (
        ii,
        pyo.value(p.model.obj),
        pyo.value(p.model.light_crude_import),
        pyo.value(p.model.heavy_crude_import),
        pyo.value(p.model.light_prod_output),
        pyo.value(p.model.med_prod_output),
        pyo.value(p.model.heavy_prod_output),
        pyo.value(p.model.light_to_light_unit),
        pyo.value(p.model.med_to_light_unit),
        pyo.value(p.model.light_to_med_unit),
        pyo.value(p.model.med_to_med_unit),
        pyo.value(p.model.med_to_heavy_unit),
        pyo.value(p.model.heavy_to_heavy_unit),
    )

In [7]:
recourse_df = pd.DataFrame(columns=non_recourse_df.columns)

In [8]:
# Create and solve the recourse model
p = PlantRecourse(
    crude_distil_cap=crude_cap,
    crude_light_ratios=crude_light_ratios,
    crude_heavy_ratios=crude_heavy_ratios,
    refine_light_cap=c,
    refine_medium_cap=c,
    refine_heavy_cap=c,
    prod_light_ratios=light_product_ratios,
    prod_medium_ratios=medium_product_input_ratios,
    prod_heavy_ratios=heavy_product_input_ratios,
    crude_light_price=crude_light_price,
    crude_heavy_price=crude_heavy_price,
    scenario_count=scenarios,
    prod_light_price=prod_light_price,
    prod_light_demand=prod_light_demand,
    prod_medium_price=prod_medium_price,
    prod_medium_demand=prod_medium_demand,
    prod_heavy_price=prod_heavy_price,
    prod_heavy_demand=prod_heavy_demand,
)

results = opt.solve(p.model)

In [9]:
# Extract the results of the recourse model into dataframe
recourse_df["Scenario"] = range(scenarios)
recourse_df["Objective Value"] = [
    scenario_obj(
        p.model,
        ii,
        prod_light_price,
        prod_medium_price,
        prod_heavy_price,
        crude_light_price,
        crude_heavy_price,
    )
    for ii in range(scenarios)
]
recourse_df["Light Crude Import"] = [
    pyo.value(p.model.light_crude_import) for ii in range(scenarios)
]
recourse_df["Heavy Crude Import"] = [
    pyo.value(p.model.heavy_crude_import) for ii in range(scenarios)
]
recourse_df["Light Product Output"] = [
    pyo.value(p.model.light_prod_output[ii]) for ii in range(scenarios)
]
recourse_df["Medium Product Output"] = [
    pyo.value(p.model.med_prod_output[ii]) for ii in range(scenarios)
]
recourse_df["Heavy Product Output"] = [
    pyo.value(p.model.heavy_prod_output[ii]) for ii in range(scenarios)
]
recourse_df["Light Inter to Light Unit"] = [
    pyo.value(p.model.light_to_light_unit[ii]) for ii in range(scenarios)
]
recourse_df["Med Inter to Light Unit"] = [
    pyo.value(p.model.med_to_light_unit[ii]) for ii in range(scenarios)
]
recourse_df["Light Inter to Med Unit"] = [
    pyo.value(p.model.light_to_med_unit[ii]) for ii in range(scenarios)
]
recourse_df["Med Inter to Med Unit"] = [
    pyo.value(p.model.med_to_med_unit[ii]) for ii in range(scenarios)
]
recourse_df["Med Inter to Heavy Unit"] = [
    pyo.value(p.model.med_to_heavy_unit[ii]) for ii in range(scenarios)
]
recourse_df["Heavy Inter to Heavy Unit"] = [
    pyo.value(p.model.heavy_to_heavy_unit[ii]) for ii in range(scenarios)
]

# Results

In [10]:
from IPython.display import display
display(non_recourse_df)
display(recourse_df)

,Scenario,Objective Value,Light Crude Import,Heavy Crude Import,Light Product Output,Medium Product Output,Heavy Product Output,Light Inter to Light Unit,Med Inter to Light Unit,Light Inter to Med Unit,Med Inter to Med Unit,Med Inter to Heavy Unit,Heavy Inter to Heavy Unit
0,0.0,8000.0,0.0,100.0,0.0,100.0,800.0,0.0,0.0,0.0,100.0,0.0,400.0
1,1.0,10000.0,0.0,100.0,50.0,0.0,850.0,0.0,50.0,0.0,0.0,50.0,400.0
2,2.0,28500.0,100.0,0.0,630.0,0.0,0.0,300.0,30.0,0.0,0.0,0.0,0.0


,Scenario,Objective Value,Light Crude Import,Heavy Crude Import,Light Product Output,Medium Product Output,Heavy Product Output,Light Inter to Light Unit,Med Inter to Light Unit,Light Inter to Med Unit,Med Inter to Med Unit,Med Inter to Heavy Unit,Heavy Inter to Heavy Unit
0,0,6075.0,25.0,75.0,0.0,157.5,600.0,0.0,0.0,75.0,82.5,0.0,300.0
1,1,8325.0,25.0,75.0,50.0,50.0,682.5,25.0,0.0,50.0,0.0,82.5,300.0
2,2,16125.0,25.0,75.0,232.5,0.0,600.0,75.0,82.5,0.0,0.0,0.0,300.0


Notably:
- The recourse model chooses to import a combination of light and heavy crude that is not optimal in any specific scenario, but provides the best overall result, making using of the ability to use the light intermediate in the medium unit in Scenario 0, where light demand is zero.
    - This can be seen as buying enough light crude to take advantage of the chance that there is demand for light product, without fully committing to it.
- The objective value of the recourse model is lower in each scenario, which is expected given the lack of "perfect foresight" the individual solutions have.